In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from models.lenet_300_100 import Real, Quat, std_hparams
import pandas as pd
import numpy as np4
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [ ]:
batch_size = std_hparams['training']['batch_size']
num_epochs = std_hparams['training']['num_epochs']
learning_rate = std_hparams['training']['learning_rate']

In [4]:
device = torch.device('cuda:0')

data = torch.tensor(pd.read_csv("../data/mnist/train.csv", header=None).values)
x, y = (data[:, 1:]/255).float().to(device), torch.nn.functional.one_hot(data[:, 0].long(), 10).to(device)

test = torch.tensor(pd.read_csv("../data/mnist/test.csv", header=None).values)
x_test, y_test = (test[:, 1:]/255).float().to(device), torch.nn.functional.one_hot(test[:, 0].long(), 10).to(device)

In [ ]:
model = Real()
model.to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
losses = []

In [ ]:
for epoch in tqdm(range(1000)):
    batch_size = 2**13
    for i in range(0, len(x), batch_size):
        batch_x, batch_y = x[i:i+batch_size], y[i:i+batch_size].float()
        optimiser.zero_grad()
        output = model(batch_x)
        loss = F.mse_loss(output, batch_y)
        loss.backward()
        optimiser.step()
    losses.append(loss.item())

In [ ]:
accuracy = (model(x).argmax(1) == y.argmax(1)).sum().item()/len(x)
print(f"Training Accuracy: {accuracy*100:.2f}%")

accuracy = (model(x_test).argmax(1) == y_test.argmax(1)).sum().item()/len(x_test)
print(f"Testing Accuracy: {accuracy*100:.2f}%")

In [ ]:
y_pred = torch.cat([model(c).detach().cpu() for c in torch.chunk(x, 10)])
y_pred_test = model(x_test).detach().cpu()
train_accuracy.append(accuracy_score(y.argmax(1).cpu(), y_pred.argmax(1))*100)
test_accuracy.append(accuracy_score(y_test.argmax(1).cpu(), y_pred_test.argmax(1))*100)